# Flux Balance Analysis (08302023 data)
The purpose of the notebook is to run flux balance analysis to find the set of genome scale fluxes that maximizes biomass production.<br><br>
The flux balance analysis results are then compared to reaction rates determined by 13C-metabolic flux analysis (13C-MFA). <br><br>
The results of this notebook will be compared to transcript constained genome scale model flux results via E-Flux2. <br><br>
This notebook looks at glucose, glycerol, acetate, and oleic acid as sole carbon sources. <br><br>
In all cases, parsimonious flux balance analysis (pFBA) was used to prevent degenerate solutions.


### Load imports

In [1]:
import cobra
import pandas as pd
import sys

source_dir = '../src'
sys.path.append(source_dir)
from add_column_to_13c_flux_df import add_column_to_13c_flux_df
from flux_prediction_scatterplot import flux_prediction_scatterplot

### Load the genome scale model

In [2]:
model = cobra.io.json.load_json_model("../genome_scale_models/iYLI647_corr_2.json")
model

Name,model
Memory address,11156ed10
Number of metabolites,1121
Number of reactions,1348
Number of genes,648
Number of groups,0
Objective expression,1.0*biomass_C - 1.0*biomass_C_reverse_c1d5c
Compartments,"c, e, m, n, x, r, g, v"


### Load 13C-MFA data

In [3]:
# load glucose 13C MFA data from ../data/13c_mfa/INCA_model_08302023_GR.xlsx
central_rxn_df = pd.read_excel('../data/13c_mfa/INCA_model_08302023_GR.xlsx', sheet_name='0830')

# drop every column after 'oleic acid'
central_rxn_df = central_rxn_df.loc[:, :'oleic acid']

# only keep the reactions that are mapped to the GSM
central_rxn_df.dropna(subset = ["reaction_ids"], inplace=True)

print(f'There are {len(central_rxn_df)} reactions in the 13C MFA that are mapped to the GSM')

central_rxn_df.head()

There are 43 reactions in the 13C MFA that are mapped to the GSM


,Unnamed: 0,ID,Equation,reaction_ids,pathway,compartment,glucose,glycerol,oleic acid
0,NaN,uptake,Glucose + ATP -> G6P,reverse_EX_glc(e),substrate_uptake,cytosol,100 ± 0,NaN,NaN
1,NaN,R3 glyc3p,GLYC + ATP -> Glyc3P,reverse_GLYCt,emp,cytosol,NaN,100 ± 0,NaN
3,NaN,OA uptake,OA + ATP -> 9*ACCOAcyt + 7*NADH + 7*FADH2,OCDCEAt,substrate_uptake,cytosol,NaN,NaN,100 ± 0
4,glycolysis/\ngluconeogensis,R4 net,G6P <-> F6P,PGI,emp,cytosol,17.8 ± 7.9,-12.3 ± 21.2,-151.7 ± 164.5
5,NaN,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,emp,cytosol,68.1 ± 2.6,-6.7 ± 6.9,-62.8 ± 53.4


### Calculate glucose GSM pFBA solution

In [4]:
# minimal medium with glucose as sole carbon source
model.medium = {
  'EX_glc(e)': 100.0, # 100 mmol of glucose
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  glucose_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in glucose_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux,
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  glucose_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  glucose_gsm_fba_df = glucose_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  glucose_gsm_fba_df = glucose_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  glucose_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/glucose_gsm_fba.csv', index=False)

glucose_gsm_fba_df.head()


100.0 EX_glc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-561.830307
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],341.609652
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],335.476108
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,326.533474
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-325.265198


### Add glucose pFBA column to 13C-MFA data

In [5]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_column_to_13c_flux_df(central_rxn_df, glucose_gsm_fba_df, 'glucose pFBA Flux')

central_rxn_df.head()

,Unnamed: 0,ID,Equation,reaction_ids,pathway,compartment,glucose,glycerol,oleic acid,glucose pFBA Flux
0,NaN,uptake,Glucose + ATP -> G6P,reverse_EX_glc(e),substrate_uptake,cytosol,100 ± 0,NaN,NaN,100.000000
1,NaN,R3 glyc3p,GLYC + ATP -> Glyc3P,reverse_GLYCt,emp,cytosol,NaN,100 ± 0,NaN,0.000000
3,NaN,OA uptake,OA + ATP -> 9*ACCOAcyt + 7*NADH + 7*FADH2,OCDCEAt,substrate_uptake,cytosol,NaN,NaN,100 ± 0,0.000000
4,glycolysis/\ngluconeogensis,R4 net,G6P <-> F6P,PGI,emp,cytosol,17.8 ± 7.9,-12.3 ± 21.2,-151.7 ± 164.5,35.574485
5,NaN,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,emp,cytosol,68.1 ± 2.6,-6.7 ± 6.9,-62.8 ± 53.4,38.629379


### Calculate glycerol GSM pFBA solution

In [6]:
# minimal medium with glycerol as sole carbon source
model.medium = {
  'EX_glyc(e)': 100.0, # 100 mmol of glycerol
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}
  
with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  glycerol_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in glycerol_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux,
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  glycerol_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  glycerol_gsm_fba_df = glycerol_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  glycerol_gsm_fba_df = glycerol_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  glycerol_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/glycerol_gsm_fba.csv', index=False)

glycerol_gsm_fba_df.head()


100.0 EX_glyc(e)
10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-574.850423
421,EX_h2o(e),H2O exchange,h2o[e] <=>,354.104943
959,H2Ot,H2O transport via diffusion,h2o[e] <=> h2o[c],-354.104943
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],315.462030
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,313.155793


### Add glycerol pFBA column to 13C-MFA data

In [7]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_column_to_13c_flux_df(central_rxn_df, glycerol_gsm_fba_df, 'glycerol pFBA Flux')

central_rxn_df.head()

,Unnamed: 0,ID,Equation,reaction_ids,pathway,compartment,glucose,glycerol,oleic acid,glucose pFBA Flux,glycerol pFBA Flux
0,NaN,uptake,Glucose + ATP -> G6P,reverse_EX_glc(e),substrate_uptake,cytosol,100 ± 0,NaN,NaN,100.000000,0.000000
1,NaN,R3 glyc3p,GLYC + ATP -> Glyc3P,reverse_GLYCt,emp,cytosol,NaN,100 ± 0,NaN,0.000000,100.000000
3,NaN,OA uptake,OA + ATP -> 9*ACCOAcyt + 7*NADH + 7*FADH2,OCDCEAt,substrate_uptake,cytosol,NaN,NaN,100 ± 0,0.000000,0.000000
4,glycolysis/\ngluconeogensis,R4 net,G6P <-> F6P,PGI,emp,cytosol,17.8 ± 7.9,-12.3 ± 21.2,-151.7 ± 164.5,35.574485,-27.963948
5,NaN,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,emp,cytosol,68.1 ± 2.6,-6.7 ± 6.9,-62.8 ± 53.4,38.629379,-18.670033


### Calculate oleic acid GSM pFBA solution

In [8]:
# minimal medium with oleic acid as sole carbon source
model.medium = {
  'EX_ocdcea(e)': 10.0, # 10 mmol of oleic acid (to prevent maxing out reactions with 1000 max flux)
  'EX_h2o(e)': 10000.0,
  'EX_h(e)': 10000.0,
  'EX_nh4(e)': 10000.0,
  'EX_o2(e)': 10000.0,
  'EX_pi(e)': 10000.0,
  'EX_so4(e)': 10000.0,
}

with model:
  # update the model to use the specified medium
  medium = model.medium
  [print(model.medium[m], m) for m in model.medium]

  # find the optimal solution
  acetate_fba_solution = cobra.flux_analysis.pfba(model)

  # make a list of dictionaries with the reaction id, name, flux, and absolute flux
  reactions = []
  for reaction_id, flux in acetate_fba_solution.fluxes.items():
    reactions.append({
      'reaction_id': reaction_id,
      'reaction_name': model.reactions.get_by_id(reaction_id).name,
      'full_reaction': model.reactions.get_by_id(reaction_id).reaction,
      'flux': flux * 10, # flux normalized to 100 mols of oleic acid
      'absolute_flux': abs(flux), # use for sorting, then drop
    })

  # make a dataframe from the list of dictionaries
  oleic_acid_gsm_fba_df = pd.DataFrame(reactions)

  # sort the dataframe by absolute flux
  oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.sort_values(by=['absolute_flux'], ascending=False)

  # drop the absolute flux column
  oleic_acid_gsm_fba_df = oleic_acid_gsm_fba_df.drop(columns=['absolute_flux'])

  # save the dataframe to a csv file
  oleic_acid_gsm_fba_df.to_csv('../results/fba_gsm_fluxes/oleic_acid_gsm_fba.csv', index=False)

oleic_acid_gsm_fba_df.head()


10000.0 EX_h2o(e)
10000.0 EX_h(e)
10000.0 EX_nh4(e)
10000.0 EX_o2(e)
10.0 EX_ocdcea(e)
10000.0 EX_pi(e)
10000.0 EX_so4(e)


,reaction_id,reaction_name,full_reaction,flux
689,H2Otm,H2O transport mitochondrial,h2o[c] <=> h2o[m],-3584.266821
607,ATPS3m,ATP synthase mitochondrial,adp[m] + 3.0 h[c] + pi[m] --> atp[m] + h2o[m] ...,2269.151708
990,ATPtm_H,ADPATP transporter mitochondrial,adp[c] + atp[m] + h[c] --> adp[m] + atp[c] + h[m],2265.438412
976,PIt2m,phosphate transporter mitochondrial,h[c] + pi[c] <=> h[m] + pi[m],2265.438412
617,CYOR_u6m,ubiquinol 6 cytochrome c reductase,2.0 ficytc[m] + 1.5 h[m] + q6h2[m] --> 2.0 foc...,2053.128358


### Add oleic acid pFBA column to 13C-MFA data

In [9]:
# add the GSM flux predictions to the 13C-MFA dataframe
central_rxn_df = add_column_to_13c_flux_df(central_rxn_df, oleic_acid_gsm_fba_df, 'oleic acid pFBA Flux')

central_rxn_df.head()

,Unnamed: 0,ID,Equation,reaction_ids,pathway,compartment,glucose,glycerol,oleic acid,glucose pFBA Flux,glycerol pFBA Flux,oleic acid pFBA Flux
0,NaN,uptake,Glucose + ATP -> G6P,reverse_EX_glc(e),substrate_uptake,cytosol,100 ± 0,NaN,NaN,100.000000,0.000000,0.000000
1,NaN,R3 glyc3p,GLYC + ATP -> Glyc3P,reverse_GLYCt,emp,cytosol,NaN,100 ± 0,NaN,0.000000,100.000000,0.000000
3,NaN,OA uptake,OA + ATP -> 9*ACCOAcyt + 7*NADH + 7*FADH2,OCDCEAt,substrate_uptake,cytosol,NaN,NaN,100 ± 0,0.000000,0.000000,100.000000
4,glycolysis/\ngluconeogensis,R4 net,G6P <-> F6P,PGI,emp,cytosol,17.8 ± 7.9,-12.3 ± 21.2,-151.7 ± 164.5,35.574485,-27.963948,-30.701311
5,NaN,R5 net,F6P + ATP <-> FBP,PFK or reverse_FBP,emp,cytosol,68.1 ± 2.6,-6.7 ± 6.9,-62.8 ± 53.4,38.629379,-18.670033,-69.437259


### Save 13C-MFA data with pFBA data added

In [12]:
# save the dataframe to a csv file
central_rxn_df.to_csv('../results/central_fluxes/pfba.csv', index=False, encoding='utf-8-sig')